In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import numbers
# import usual libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import f1_score
import pickle as p
# our system imports
from aideme.active_learning import *
from aideme.initial_sampling import StratifiedSampler
from aideme.explore import *
from aideme.io import read_task
from aideme.utils import *
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import sklearn
from sklearn.cluster import KMeans

%matplotlib inline

In [2]:
dataset = p.load(open("kaggle_jobs_dataset.p",'rb'))
# Non-Decreasing categorization for Salary
salary_liste = list(dataset.Queried_Salary.unique())
salary_category = {}
for cate, i in enumerate(salary_liste):
    salary_category[i] = cate
dataset['Salary'] = dataset.Queried_Salary.apply(lambda x: salary_category[x]) 

dataset = dataset.join(pd.get_dummies(dataset.Job_Type)).drop('Job_Type',1)

### CREATE A SET OF SKILLS , ONE COLUMN PER SKILL
minimum_appearances = 100
def g(x):
    answer = []
    for i in x:
        if i in liste_skills:
            answer.append(i)
    return answer

f = lambda x :[] if str(x)=='nan' else x.replace('[','').replace(']','').replace('"','').replace(" ","").replace("'",'').lower().split(',') 

skill_count = pd.Series(dataset.Skill.apply(f).sum()).value_counts()
liste_skills = list(skill_count[skill_count>minimum_appearances].index)
dataset['new_skill'] = pd.Series(dataset.Skill.apply(f)).apply(g)
for col in liste_skills:
    dataset[col] = dataset['new_skill'].apply(lambda x: int(col in x))
### SET OF SKILLS DONE

import pickle as p
#bert = p.load(open('representation_bert.p','rb'))
#dataset['bert'] = bert

# bert_1 = p.load(open('representation_bert_1000.p','rb'))
# bert_2 = p.load(open('representation_bert_2000.p','rb'))
# bert_3 = p.load(open('representation_bert_3000.p','rb'))
# bert_4 = p.load(open('representation_bert_4000.p','rb'))
# bert_5 = p.load(open('representation_bert_end_last.p','rb'))
# bert_full_rep = bert_1+bert_2+bert_3+bert_4+bert_5
# bert_full_rep = [x.reshape(-1,1024).mean(0) for x in bert_full_rep]

bert = p.load(open('representation_bert.p','rb'))
dataset['bert'] = bert
df_bert = pd.DataFrame(np.concatenate(np.array(bert)).reshape(len(bert), -1))

### JOB TITLE DATAFRAME
jobtitle = p.load(open('representation_bert_jobtitle.p','rb'))
df_jobtitle = pd.DataFrame(np.array(jobtitle).reshape(-1,1024))

In [3]:
cols = [col for col in dataset.columns if (dataset.isna().sum()<100)[col]]
cols = [col for col in cols if (isinstance(dataset[col][0], numbers.Number))]
X = dataset[cols]

y = np.logical_and.reduce([X['data_engineer']>0]) #, X['data_analyst']>0, X['No_of_Skills']>2])
print('number of valid elements:', str(y.sum())+',','proportion', str(y.sum()*100/len(y))+'%')
print('number of columns:', len(X.columns))

number of valid elements: 1379, proportion 24.129483814523184%
number of columns: 103


In [4]:
#### DEFINITIONS OF THE DIFFERENTS INTEREST SETS

y_luciano = [1774, 1786, 1800, 1870, 1994, 2002, 2048, 2060, 2137, 2164, 2227, 2229, 2449, 2455, 2473]
y_enhui = [ 994, 1063, 1134, 1836, 1856, 1857, 1858, 1859, 1865, 1908, 1975,
       2054, 2285, 2312]
y_bastien = (p.load(open('myjobdata.p','rb')))
y_bastien = list(y_bastien[y_bastien['value']==1].index)
y_pierre = p.load(open('pierre_label.p','rb'))
b = (np.array([np.array(tuple_) for tuple_ in y_pierre]).reshape(-1,2))
y_pierre = [x[0] for x in b if x[1]==1]
df_chiheb = pd.read_csv('chiheb_annotation.csv')
b = dataset.merge(df_chiheb)['annotation_chiheb']
y_chiheb = list(b[b==1].index)
df_marc = pd.read_csv('marc_annotation.csv')
b = dataset.merge(df_marc)['marc_annotation_final']
y_marc = list(b[b=="*"].index)
y_benoit = [47, 162, 256, 276, 370, 876, 881,  897,  928, 960,  974, 984, 985, 990,  1054, 1113, 1738, 1747, 1757,  1764, 2200, 2228, 2309, 2349, 2351, 2371, 2374, 2403, 2471, 2514, 2534]

In [5]:
X['index_'] = list(X.index)
y_bas = X['index_'].apply(lambda x : x in y_bastien)
y_luc = X['index_'].apply(lambda x : x in y_luciano)
y_enh = X['index_'].apply(lambda x : x in y_enhui)
y_pie = X['index_'].apply(lambda x : x in y_pierre)
y_chi = X['index_'].apply(lambda x : x in y_chiheb)
y_mar = X['index_'].apply(lambda x : x in y_marc)
y_ben = X['index_'].apply(lambda x : x in y_benoit)
X = X.drop(['index_'],1)

/home/bastien/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [6]:
# SET-UP EXPLORATION CONFIGURATION
import warnings
warnings.filterwarnings("ignore")
NUMBER_OF_ITERATIONS = 170  # number of points to be labeled by the user
SUBSAMPLING = float('inf') #10000, 50000, float('inf')
INITIAL_SAMPLER = StratifiedSampler(pos=1, neg=1)  # start with one random positive sample and one random negative sample
CALLBACK = [classification_metrics('fscore','true_positive','false_positive','false_negative','true_negative')]  # callback function to be called at the end of each iteration. Here, we compute the current f-score
CALLBACK_SKIP = 10
PRINT_CALLBACK_RESULT = True
explore = PoolBasedExploration(NUMBER_OF_ITERATIONS, INITIAL_SAMPLER, SUBSAMPLING, CALLBACK, CALLBACK_SKIP, PRINT_CALLBACK_RESULT)

# CHOOSE AN ALGORITHM
#learner = RandomSampler(SVC(C=1e5, kernel='rbf'))  # choose a random point
learner1 = SimpleMargin(C=1e5,  kernel='rbf')
learner3 = KernelQueryByCommittee(kernel='rbf', n_samples=8, warmup=10, thin=5, sampling='deterministic')
learner2 = DualSpaceModel(learner1, sample_unknown_proba=0.1)
learner4 = DualSpaceModel(learner3, sample_unknown_proba=0.1)
# RUN EXPLORATION
metrics = explore.run(X, y_ben, learner1, repeat=3)  # 'repeat' specifies how many times to repeat the exploration process
metrics2 = explore.run(X, y_ben, learner2, repeat=3)

# with text
X_one = .join(df_bert)
metrics3 = explore.run(X_one, y_ben, learner1, repeat=3)  # 'repeat' specifies how many times to repeat the exploration process
metrics4 = explore.run(X_one, y_pie, learner2, repeat=3)

# with jobtitle 
X_second = X_one.join(df_jobtitle, lsuffix='lol')
metrics5 = explore.run(X_second, y_ben, learner1, repeat=3)  # 'repeat' specifies how many times to repeat the exploration process
metrics6 = explore.run(X_second, y_ben, learner2, repeat=3)

# with both
# with jobtitle 

metrics7 = explore.run(X_second, y_ben, learner3, repeat=3)  # 'repeat' specifies how many times to repeat the exploration process
metrics8 = explore.run(X_second, y_ben, learner4, repeat=3)

# COMPUTE AVERAGE F-SCORE OVER ALL REPEATS AND PLOT
df_list = [pd.DataFrame.from_dict({i: metric for i, metric in enumerate(ls)}, orient='index') for ls in metrics]
avg = sum([df['fscore'][~df['fscore'].isna()] for df in df_list]) / len(df_list)
avg.plot(ylim=[0,1], marker='o',label='SVM - Data Only')

df_list2 = [pd.DataFrame.from_dict({i: metric for i, metric in enumerate(ls)}, orient='index') for ls in metrics2]
avg2 = sum([df['fscore'][~df['fscore'].isna()] for df in df_list2]) / len(df_list2)
avg2.plot(ylim=[0,1], marker='o',label='DSM Data Only')

df_list3 = [pd.DataFrame.from_dict({i: metric for i, metric in enumerate(ls)}, orient='index') for ls in metrics3]
avg3 = sum([df['fscore'][~df['fscore'].isna()] for df in df_list3]) / len(df_list3)
avg3.plot(ylim=[0,1], marker='o',label='SVM Data -Text Only')

df_list4 = [pd.DataFrame.from_dict({i: metric for i, metric in enumerate(ls)}, orient='index') for ls in metrics4]
avg4 = sum([df['fscore'][~df['fscore'].isna()] for df in df_list4]) / len(df_list4)
avg4.plot(ylim=[0,1], marker='o',label='DSM Text-Data Only')

df_list5 = [pd.DataFrame.from_dict({i: metric for i, metric in enumerate(ls)}, orient='index') for ls in metrics5]
avg5 = sum([df['fscore'][~df['fscore'].isna()] for df in df_list5]) / len(df_list5)
avg5.plot(ylim=[0,1], marker='o',label='SVM All included')

df_list6 = [pd.DataFrame.from_dict({i: metric for i, metric in enumerate(ls)}, orient='index') for ls in metrics6]
avg6 = sum([df['fscore'][~df['fscore'].isna()] for df in df_list6]) / len(df_list6)
avg6.plot(ylim=[0,1], marker='o',label='DSM All included')

df_list7 = [pd.DataFrame.from_dict({i: metric for i, metric in enumerate(ls)}, orient='index') for ls in metrics7]
avg7 = sum([df['fscore'][~df['fscore'].isna()] for df in df_list7]) / len(df_list7)
avg7.plot(ylim=[0,1], marker='o',label='KbC All included')

df_list8 = [pd.DataFrame.from_dict({i: metric for i, metric in enumerate(ls)}, orient='index') for ls in metrics8]
avg8 = sum([df['fscore'][~df['fscore'].isna()] for df in df_list8]) / len(df_list8)
avg8.plot(ylim=[0,1], marker='o',label='DSM-KbC All included')

plt.show()

idx [1764, 2787]
X  [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 4 0 0 1 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [3 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]] (2, 103)
labels  [ True False]
data (array([], shape=(0, 103), dtype=int64), array([], dtype=float64))
iter: 0, fscore: 0.026272577996715927, true_positive: 8, false_positive: 570, false_negative: 23, true_negative: 5114
idx [5639]
X  [[6 0 0 1 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 5 0 1 0 1 0 0 1
  0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]] (1, 103)
data (array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0

idx [4199]
X  [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 3 1 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]] (1, 103)
data (array([[0, 0, 0, ..., 0, 0, 0],
       [3, 1, 0, ..., 0, 0, 0],
       [6, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False]))
idx [2534]
X  [[1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 5 0 0 1 1 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]] (1, 103)
labels  [ True]
data (array([[0, 0, 0, ..., 0, 0, 0],
       [3, 1, 0, ..., 0, 0, 0],
       [6, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
      

idx [1885]
X  [[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 4 0 0 1 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]] (1, 103)
data (array([[0, 0, 0, ..., 0, 0, 0],
       [3, 1, 0, ..., 0, 0, 0],
       [6, 0, 0, ..., 0, 0, 0],
       ...,
       [2, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]]), array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False]))
idx [1822]
X  [[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 4 0 0 1 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]] (1, 103)
data

idx [2360]
X  [[2 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 5 0 0 1 1 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]] (1, 103)
data (array([[0, 0, 0, ..., 0, 0, 0],
       [3, 1, 0, ..., 0, 0, 0],
       [6, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [3, 0, 0, ..., 0, 0, 0],
       [3, 0, 0, ..., 0, 0, 0]]), array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False]))
idx [1765]
X  [[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 4 0 0 1 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

idx [4319]
X  [[1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 5 1 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]] (1, 103)
data (array([[0, 0, 0, ..., 0, 0, 0],
       [3, 1, 0, ..., 0, 0, 0],
       [6, 0, 0, ..., 0, 0, 0],
       ...,
       [3, 1, 1, ..., 0, 0, 0],
       [2, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]]), array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False]))
iter: 60, fscore: 0.12121212121212122, true_positive: 2, false_positive: 

...

idx [2714]
X  [[2 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]] (1, 103)
data (array([[3, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [3, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False

idx [2926]
X  [[2 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]] (1, 103)
data (array([[3, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [3, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False

idx [3]
X  [[1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]] (1, 103)
data (array([[3, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [3, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [3, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]]), array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, F

iter: 140, fscore: 0.12121212121212122, true_positive: 2, false_positive: 0, false_negative: 29, true_negative: 5684
idx [2809]
X  [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]] (1, 103)
data (array([[3, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [3, 0, 0, ..., 0, 0, 0],
       ...,
       [3, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, 

KeyboardInterrupt: 

In [ ]:
# COMPUTE AVERAGE F-SCORE OVER ALL REPEATS AND PLOT
df_list = [pd.DataFrame.from_dict({i: metric for i, metric in enumerate(ls)}, orient='index') for ls in metrics]
avg = sum([df['fscore'][~df['fscore'].isna()] for df in df_list]) / len(df_list)
avg.plot(ylim=[0,1], marker='o',label='SVM - Data Only')

df_list2 = [pd.DataFrame.from_dict({i: metric for i, metric in enumerate(ls)}, orient='index') for ls in metrics2]
avg2 = sum([df['fscore'][~df['fscore'].isna()] for df in df_list2]) / len(df_list2)
avg2.plot(ylim=[0,1], marker='o',label='DSM Data Only')

df_list3 = [pd.DataFrame.from_dict({i: metric for i, metric in enumerate(ls)}, orient='index') for ls in metrics3]
avg3 = sum([df['fscore'][~df['fscore'].isna()] for df in df_list3]) / len(df_list3)
avg3.plot(ylim=[0,1], marker='o',label='SVM Text-Data')

df_list4 = [pd.DataFrame.from_dict({i: metric for i, metric in enumerate(ls)}, orient='index') for ls in metrics4]
avg4 = sum([df['fscore'][~df['fscore'].isna()] for df in df_list4]) / len(df_list4)
avg4.plot(ylim=[0,1], marker='o',label='DSM Text-Data')

df_list5 = [pd.DataFrame.from_dict({i: metric for i, metric in enumerate(ls)}, orient='index') for ls in metrics5]
avg5 = sum([df['fscore'][~df['fscore'].isna()] for df in df_list5]) / len(df_list5)
avg5.plot(ylim=[0,1], marker='o',label='SVM All included')

df_list6 = [pd.DataFrame.from_dict({i: metric for i, metric in enumerate(ls)}, orient='index') for ls in metrics6]
avg6 = sum([df['fscore'][~df['fscore'].isna()] for df in df_list6]) / len(df_list6)
avg6.plot(ylim=[0,1], marker='o',label='DSM All included')

df_list7 = [pd.DataFrame.from_dict({i: metric for i, metric in enumerate(ls)}, orient='index') for ls in metrics7]
avg7 = sum([df['fscore'][~df['fscore'].isna()] for df in df_list7]) / len(df_list7)
avg7.plot(ylim=[0,1], marker='o',label='KbC All included')

df_list8 = [pd.DataFrame.from_dict({i: metric for i, metric in enumerate(ls)}, orient='index') for ls in metrics8]
avg8 = sum([df['fscore'][~df['fscore'].isna()] for df in df_list8]) / len(df_list8)
avg8.plot(ylim=[0,1], marker='o',label='DSM-KbC All included')

plt.legend()
plt.savefig('test_ben_bis.png')
plt.show()

In [ ]:
# SET-UP EXPLORATION CONFIGURATION
NUMBER_OF_ITERATIONS = 150  # number of points to be labeled by the user
SUBSAMPLING = float('inf') #10000, 50000, float('inf')
INITIAL_SAMPLER = StratifiedSampler(pos=1, neg=1)  # start with one random positive sample and one random negative sample
CALLBACK = [classification_metrics('fscore'), three_set_metric]  # callback function to be called at the end of each iteration. Here, we compute the current f-score
CALLBACK_SKIP = 5
PRINT_CALLBACK_RESULT = True

explore = PoolBasedExploration(NUMBER_OF_ITERATIONS, INITIAL_SAMPLER, SUBSAMPLING, CALLBACK, CALLBACK_SKIP, PRINT_CALLBACK_RESULT)

# SUBSPATIAL LEARNER - run Simple Margin in each subspace
# subspatial_active_learners = [
#     KernelQueryByCommittee(kernel='rbf', n_samples=8, warmup=100, thin=10, sampling='deterministic'),
#     KernelQueryByCommittee(kernel='rbf', n_samples=8, warmup=100, thin=10, sampling='deterministic'),
# ]
subspatial_active_learners = [
    SimpleMargin(C=1e5,  kernel='rbf'),
    SimpleMargin(C=1e5,  kernel='rbf')]


learner = SubspaceLearner(
    partition=[[0], [1]],  # partition of attributes (one subspace with the attribute 0, and another with 1)
    learners=subspatial_active_learners, 
    #label_function='AND', probability_function='min', ranking_function='square'
)

learner = DualSpaceModel(SimpleMargin(C=1024, kernel='rbf'), sample_unknown_proba=0.5, partition=[[0], [1]])  # Dual Space model

y_subspace = np.vstack([y_mar for x in range(2)]).T.astype('float')
# RUN EXPLORATION

metrics = explore.run(X.values, y_subspace, learner, repeat=1)  # use y_subspace here

# COMPUTE AVERAGE F-SCORE OVER ALL REPEATS AND PLOT
df_list = [pd.DataFrame.from_dict({i: metric for i, metric in enumerate(ls)}, orient='index') for ls in metrics]
avg = sum([df['fscore'][~df['fscore'].isna()] for df in df_list]) / len(df_list)
avg.plot(ylim=[0,1], marker='o')
plt.show()